In [ ]:
!pip install transformers
!pip install logger
!pip install ekphrasis
!pip install optuna
!pip install datasets

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

project_dir = 'gdrive/My Drive/HLT_proj/'

Mounted at /content/gdrive


Download AlBERTo-it from GitHub and fix the tokenizer.py

In [ ]:
!git clone https://github.com/marcopoli/AlBERTo-it alberto
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wdAtBa9I3zRCdS8EBP7lUkAMBdFfAb4W' -O alberto/tokenizer.py

Cloning into 'alberto'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 169 (delta 28), reused 4 (delta 1), pack-reused 120
Receiving objects: 100% (169/169), 683.98 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
from alberto.tokenizer import *
import tensorflow as tf
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import BertTokenizer

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
[02/Sep/2023 13:45:18] INFO - NumExpr defaulting to 2 threads.


In [ ]:
alberto_preprocessor = AlBERTo_Preprocessing(do_lower_case=True)

alberto_model_name = "m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0"

alberto_tokenizer = AutoTokenizer.from_pretrained(alberto_model_name)

max_length = 250

In [ ]:
bert_model_name = "dbmdz/bert-base-italian-xxl-cased"

bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
import random

random.seed(42)

def group_tweets(df, preprocessor, bert_tokenizer, alberto_tokenizer, max_tokens = 510):
  new_data = []
  users = list(df['label'].unique())
  for user in users:
    user_tweets = df[df['label'].isin([user])]
    tweets = user_tweets['tweet'].values
    random.shuffle(tweets)
    num_tokens = 0
    samples = []
    for tweet in tweets:
      max_tokens_tweet = max([len(bert_tokenizer.tokenize(tweet)), len(alberto_tokenizer.tokenize(preprocessor.preprocess(tweet)))])

      if (num_tokens + max_tokens_tweet > max_tokens):
        tweet_sample = ' '.join(samples)
        new_data.append({'label': user, 'gender': user_tweets['gender'].values[0], 'ideology_binary': user_tweets['ideology_binary'].values[0], 'ideology_multiclass': user_tweets['ideology_multiclass'].values[0], 'tweet': tweet_sample})
        num_tokens = max_tokens_tweet
        samples = [tweet]
      else:
        num_tokens = num_tokens + max_tokens_tweet
        samples.append(tweet)

    if num_tokens > 0:
      tweet_sample = ' '.join(samples)
      new_data.append({'label': user, 'gender': user_tweets['gender'].values[0], 'ideology_binary': user_tweets['ideology_binary'].values[0], 'ideology_multiclass': user_tweets['ideology_multiclass'].values[0], 'tweet': tweet_sample})
  return new_data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

raw_data = {}

for portion in ('train', 'test'):
  raw_data[portion] = pd.read_csv(project_dir+portion+'.csv')
  raw_data[portion]['tweet'].fillna(" ", inplace=True)

###Italian Bert preprocessing e tokenization

In [ ]:
data = {'bert': {}, 'alberto': {}}

for portion in ('train', 'test'):
  #data['alberto'][portion] = data['bert'][portion] = pd.DataFrame.from_dict(group_tweets(raw_data[portion], alberto_preprocessor, bert_tokenizer, alberto_tokenizer, max_tokens = 248))
  data['bert'][portion] = pd.DataFrame.from_dict(group_tweets(raw_data[portion], alberto_preprocessor, bert_tokenizer, alberto_tokenizer, max_tokens = 248))
  data['alberto'][portion] = data['bert'][portion].copy()

In [ ]:
valid_portion = 0.1
for encoder in ('bert', 'alberto'):
  data[encoder]['train'], data[encoder]['valid'] = train_test_split(data[encoder]['train'], test_size=valid_portion, shuffle=True, random_state=42)

In [ ]:
map_gender = {'male':0, 'female':1}
map_ideology = {'left':0, 'right':1}
map_ideology_multi = {'left':0, 'right':1, 'moderate_left':2, 'moderate_right':3}
#for portion in ('train', 'test', 'valid', 'grid_train', 'grid_valid'):
for encoder in ('bert', 'alberto'):
  for portion in ('train', 'test', 'valid'):
    data[encoder][portion]['gender'] = data[encoder][portion]['gender'].map(map_gender)
    data[encoder][portion]['ideology_binary'] = data[encoder][portion]['ideology_binary'].map(map_ideology)
    data[encoder][portion]['ideology_multiclass'] = data[encoder][portion]['ideology_multiclass'].map(map_ideology_multi)
    data[encoder][portion].rename(columns = {'label':'user'}, inplace = True)
    #data[encoder][portion] = data[encoder][portion].drop('label', axis=1)

In [ ]:
#preprocessing (for aberto) before transforming in Dataset
for portion in ('train', 'test', 'valid'):
  data['alberto'][portion]['preprocces_tweet'] = data['alberto'][portion]['tweet'].apply(lambda t: alberto_preprocessor.preprocess(t))

In [ ]:
from datasets import Dataset

dataset = {'bert': {}, 'alberto': {}}

for encoder in ('bert', 'alberto'):
  for portion in ('train', 'test', 'valid'):
    dataset[encoder][portion] = Dataset.from_pandas(data[encoder][portion])

In [ ]:
for portion in ('train', 'test', 'valid'):
  dataset['alberto'][portion] = dataset['alberto'][portion].map(lambda e: alberto_tokenizer(e['preprocces_tweet'], max_length=max_length, padding='max_length'), batched=True)
  dataset['bert'][portion] = dataset['bert'][portion].map(lambda e: bert_tokenizer(e['tweet'], max_length=max_length, padding='max_length'), batched=True)

Map:   0%|          | 0/21314 [00:00<?, ? examples/s]

Map:   0%|          | 0/21314 [00:00<?, ? examples/s]

Map:   0%|          | 0/8134 [00:00<?, ? examples/s]

Map:   0%|          | 0/8134 [00:00<?, ? examples/s]

Map:   0%|          | 0/2369 [00:00<?, ? examples/s]

Map:   0%|          | 0/2369 [00:00<?, ? examples/s]

In [ ]:
for encoder in ('bert', 'alberto'):
  for portion in ('train', 'test', 'valid'):
    dataset[encoder][portion] = dataset[encoder][portion].rename_column('input_ids', encoder+'_input_ids')
    dataset[encoder][portion] = dataset[encoder][portion].rename_column('token_type_ids', encoder+'_token_type_ids')
    dataset[encoder][portion] = dataset[encoder][portion].rename_column('attention_mask', encoder+'_attention_mask')

In [ ]:
ds = {}

for portion in ('train', 'test', 'valid'):
  ds[portion] = dataset['bert'][portion].add_column('alberto_input_ids', dataset['alberto'][portion]['alberto_input_ids'])
  ds[portion] = ds[portion].add_column('alberto_token_type_ids', dataset['alberto'][portion]['alberto_token_type_ids'])
  ds[portion] = ds[portion].add_column('alberto_attention_mask', dataset['alberto'][portion]['alberto_attention_mask'])

Save to disk

In [ ]:
for portion in ('train', 'test', 'valid'):
  ds[portion].save_to_disk(project_dir + 'ds_' + portion)

Saving the dataset (0/1 shards):   0%|          | 0/21305 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8126 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2368 [00:00<?, ? examples/s]

Load from disk

In [ ]:
from datasets import load_from_disk

ds = {}

for portion in ('train', 'test', 'valid'):
  ds[portion] = load_from_disk(project_dir + 'ds_' + portion)

In [ ]:
from transformers import DataCollatorWithPadding

#data_collator = {}
#data_collator['bert'] = DataCollatorWithPadding(tokenizer=bert_tokenizer, return_tensors="tf")
#data_collator['alberto'] = DataCollatorWithPadding(tokenizer=alberto_tokenizer, return_tensors="tf")

tf_datasets = {}

for portion in ('train', 'test', 'valid'):
  tf_datasets[portion] = ds[portion].to_tf_dataset(
  columns=['bert_input_ids', 'bert_token_type_ids', 'bert_attention_mask',
           'alberto_input_ids', 'alberto_token_type_ids', 'alberto_attention_mask'],
  label_cols = ['gender', 'ideology_binary', 'ideology_multiclass'],
  shuffle=False,
  batch_size=32,
  #collate_fn=data_collator[encoder],
  )

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import plot_model
from transformers import TFAutoModel, BertConfig, TFBertModel

## Optuna Grid-Search

In [ ]:
from statistics import mode
from sklearn.metrics import f1_score, classification_report
import numpy as np

def f1(real_df, predicted_df):
  f1_scores = {}

  for label in ['gender', 'ideology_binary', 'ideology_multiclass']:
    f1_scores[label] = f1_score(real_df[label], predicted_df[label], average='macro')
    print(classification_report(real_df[label], predicted_df[label]))

  f1_scores = list(f1_scores.values())

  avg_f1 = (sum(f1_scores) / len(f1_scores))

  return avg_f1

def evaluate_model_optuna(model, tf_dataset, pd_dataset):

  y_pred = model.predict(tf_dataset)

  predictions = [np.zeros(len(y_pred[0]), dtype=int), np.zeros(len(y_pred[0]), dtype=int), np.zeros(len(y_pred[0]), dtype=int)]

  for i in range(len(y_pred[0])):
    predictions[0][i] = round(y_pred[0][i][0])
    predictions[1][i] = round(y_pred[1][i][0])
    predictions[2][i] = np.argmax(y_pred[2][i])


  pd_dataset['pred_gender'] = predictions[0]
  pd_dataset['pred_ideology_binary'] = predictions[1]
  pd_dataset['pred_ideology_multiclass'] = predictions[2]

  for label in ('pred_gender', 'pred_ideology_binary', 'pred_ideology_multiclass'):
    pd_dataset[label] = pd_dataset.groupby('user')[label] \
        .transform(lambda x: x.value_counts().idxmax())

  pd_dataset.drop_duplicates(subset=['user'], inplace=True)

  predicted = pd.DataFrame()
  predicted[['gender', 'ideology_binary', 'ideology_multiclass']] = pd_dataset[['pred_gender', 'pred_ideology_binary', 'pred_ideology_multiclass']]

  avg_f1 = f1(pd_dataset, predicted)

  print(f'average f1: {avg_f1}')

  return avg_f1

In [ ]:
import logging
import sys

import optuna

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "f1_grid"  # Unique identifier of the study.
#storage_name = "sqlite:///{}.db".format(study_name)
name = 'f1_italian_bert_alberto'
storage = f"sqlite:///gdrive/My Drive/HLT_proj/optuna/{name}.db"
study = optuna.create_study(study_name=study_name, storage=storage, direction="maximize", load_if_exists=True)

[I 2023-09-01 14:34:48,013] Using an existing study with name 'f1_grid' instead of creating a new one.


Using an existing study with name 'f1_grid' instead of creating a new one.


In [ ]:
bert_dir = project_dir + 'italian_bert'
alberto_dir = project_dir + 'alberto'

def objective(trial):
  ## hyperparameters:

  units = trial.suggest_categorical('units', [300, 600])

  dropout_rate = trial.suggest_categorical('dropout_rate', [0.1, 0.3])

  learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-5])

  epochs = 3

  bert_config = BertConfig.from_pretrained(bert_dir)
  bert_encoder = TFBertModel.from_pretrained(bert_dir, config=bert_config, trainable=False)

  alberto_config = BertConfig.from_pretrained(alberto_dir)
  alberto_encoder = TFAutoModel.from_pretrained(alberto_dir, config=alberto_config, trainable=False)

  input_ids_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_input_ids')
  token_type_ids_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_token_type_ids')
  attention_mask_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_attention_mask')
  bert_embedding = bert_encoder(
      input_ids_BERT, token_type_ids=token_type_ids_BERT, attention_mask=attention_mask_BERT)['pooler_output']

  bert_embedding = layers.Dropout(dropout_rate)(bert_embedding)


  input_ids_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_input_ids')
  token_type_ids_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_token_type_ids')
  attention_mask_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_attention_mask')
  alberto_embedding = alberto_encoder(
      input_ids_ALBERTO, token_type_ids=token_type_ids_ALBERTO, attention_mask=attention_mask_ALBERTO)['pooler_output']

  alberto_embedding = layers.Dropout(dropout_rate)(alberto_embedding)

  embedding = layers.Concatenate()([bert_embedding, alberto_embedding])

  gender = layers.Dense(units=units)(embedding)
  gender = layers.Activation("relu")(gender)
  gender = layers.Dropout(dropout_rate)(gender)

  ideology = layers.Dense(units=units)(embedding)
  ideology = layers.Activation("relu")(ideology)
  ideology = layers.Dropout(dropout_rate)(ideology)

  multi_ideology = layers.Dense(units=units)(embedding)
  multi_ideology = layers.Activation("relu")(multi_ideology)
  multi_ideology = layers.Dropout(dropout_rate)(multi_ideology)

  gender_output = layers.Dense(1,  activation='sigmoid', name = 'gender')(gender)
  ideology_binary_output = layers.Dense(1,  activation='sigmoid', name = 'ideology_binary')(ideology)
  ideology_multiclass_output = layers.Dense(4,  activation='softmax', name = 'ideology_multiclass')(multi_ideology)

  input_dict = {'bert_input_ids': input_ids_BERT, 'bert_token_type_ids': token_type_ids_BERT, 'bert_attention_mask': attention_mask_BERT,
                'alberto_input_ids': input_ids_ALBERTO, 'alberto_token_type_ids': token_type_ids_ALBERTO, 'alberto_attention_mask': attention_mask_ALBERTO}


  model = keras.Model(
      inputs= input_dict,
      outputs=[gender_output, ideology_binary_output, ideology_multiclass_output]
      )


  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

  loss = {'gender': 'binary_crossentropy', 'ideology_binary': 'binary_crossentropy', 'ideology_multiclass': 'sparse_categorical_crossentropy'}

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

  history = model.fit(tf_datasets['train'],
          epochs = epochs,
          verbose = 1,
          batch_size = 32,
          validation_data = tf_datasets['valid'],
          callbacks=[callback]
                    )

  return evaluate_model_optuna(model, tf_datasets['valid'], data['bert']['valid'].copy())

In [ ]:
study.optimize(objective, n_trials=8)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/italian_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/alberto.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
667/667 [==============================] - 900s 1s/step - loss: 0.8062 - gender_loss: 0.3148 - ideology_binary_loss: 0.1525 - ideology_multiclass_loss: 0.3388 - gender_accuracy: 0.8617 - ideology_binary_accuracy: 0.9414 - ideology_multiclass_accuracy: 0.8786 - val_loss: 0.6331 - val_gender_loss: 0.2741 - val_ideology_binary_loss: 0.1178 - val_ideology_multiclass_loss: 0.2411 - val_gender_accuracy: 0.8843 - val_ideology_binary_accuracy: 0.9523 - val_ideology_multiclass_accuracy: 0.9118
Epoch 2/3
667/667 [==============================] - 863s 1s/step - loss: 0.7091 - gender_loss: 0.2838 - ideology_binary_loss: 0.1401 - ideology_multiclass_loss: 0.2852 - gender_accuracy: 0.8776 - ideology_binary_accuracy: 0.9456 - ideology_multiclass_accuracy: 0.8982 - val_loss: 0.6257 - val_gender_loss: 0.2720 - val_ideology_binary_loss: 0.1131 - val_ideology_multiclass_loss: 0.2406 - val_gender_accuracy: 0.8814 - val_ideology_binary_accuracy: 0.9531 - val_ideology_multiclass_accuracy: 0.9135


[I 2023-09-01 09:26:31,706] Trial 1 finished with value: 0.9231820063460233 and parameters: {'units': 300, 'dropout_rate': 0.1, 'learning_rate': 5e-05}. Best is trial 1 with value: 0.9231820063460233.


              precision    recall  f1-score   support

           0       0.92      0.90      0.91       686
           1       0.85      0.88      0.86       422

    accuracy                           0.89      1108
   macro avg       0.89      0.89      0.89      1108
weighted avg       0.89      0.89      0.89      1108

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       629
           1       0.98      0.94      0.96       479

    accuracy                           0.96      1108
   macro avg       0.96      0.96      0.96      1108
weighted avg       0.96      0.96      0.96      1108

              precision    recall  f1-score   support

           0       0.88      0.94      0.91       135
           1       0.96      0.93      0.94       371
           2       0.95      0.96      0.95       494
           3       0.91      0.84      0.87       108

    accuracy                           0.94      1108
   macro avg       0

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/italian_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/alberto.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
667/667 [==============================] - 902s 1s/step - loss: 0.8349 - gender_loss: 0.3201 - ideology_binary_loss: 0.1593 - ideology_multiclass_loss: 0.3556 - gender_accuracy: 0.8599 - ideology_binary_accuracy: 0.9372 - ideology_multiclass_accuracy: 0.8703 - val_loss: 0.6304 - val_gender_loss: 0.2764 - val_ideology_binary_loss: 0.1147 - val_ideology_multiclass_loss: 0.2392 - val_gender_accuracy: 0.8835 - val_ideology_binary_accuracy: 0.9531 - val_ideology_multiclass_accuracy: 0.9126
Epoch 2/3
144/667 [=====>........................] - ETA: 10:14 - loss: 0.7338 - gender_loss: 0.2903 - ideology_binary_loss: 0.1464 - ideology_multiclass_loss: 0.2972 - gender_accuracy: 0.8737 - ideology_binary_accuracy: 0.9431 - ideology_multiclass_accuracy: 0.8945

## F1-Score Evaluation After Fine-Tuning

In [ ]:
from statistics import mode
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

def f1(real_df, predicted_df):
  f1_scores = {}

  for label in ['gender', 'ideology_binary', 'ideology_multiclass']:
    f1_scores[label] = f1_score(real_df[label], predicted_df[label], average='macro')
    print(classification_report(real_df[label], predicted_df[label]))


  return f1_scores
  """f1_scores = list(f1_scores.values())

  avg_f1 = (sum(f1_scores) / len(f1_scores))

  return avg_f1"""

In [ ]:
# cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
import matplotlib.pyplot as plt

labels = {}
labels['gender'] = ['male', 'female']
labels['ideology_binary'] = ['left', 'right']
labels['ideology_multiclass'] = ['left', 'moderate_left', 'moderate_right', 'right']

def confusion(real_df, predicted_df):
  for label in ['gender', 'ideology_binary', 'ideology_multiclass']:
    cm = confusion_matrix(real_df[label], predicted_df[label], labels=labels[label])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels[label])
    disp.plot(cmap=plt.cm.Blues)
    disp.plot()
    plt.show()

In [ ]:


def evaluate_model(model, tf_dataset, pd_dataset):

  y_ped = model.predict(tf_dataset)

  predictions = [np.zeros(len(y_pred[0]), dtype=int), np.zeros(len(y_pred[0]), dtype=int), np.zeros(len(y_pred[0]), dtype=int)]

  for i in range(len(y_pred[0])):
    predictions[0][i] = round(y_pred[0][i][0])
    predictions[1][i] = round(y_pred[1][i][0])
    predictions[2][i] = np.argmax(y_pred[2][i])


  pd_dataset['pred_gender'] = predictions[0]
  pd_dataset['pred_ideology_binary'] = predictions[1]
  pd_dataset['pred_ideology_multiclass'] = predictions[2]

  for label in ('pred_gender', 'pred_ideology_binary', 'pred_ideology_multiclass'):
    pd_dataset[label] = pd_dataset.groupby('user')[label] \
        .transform(lambda x: x.value_counts().idxmax())

  pd_dataset.drop_duplicates(subset=['user'], inplace=True)

  predicted = pd.DataFrame()
  predicted[['gender', 'ideology_binary', 'ideology_multiclass']] = pd_dataset[['pred_gender', 'pred_ideology_binary', 'pred_ideology_multiclass']]

  f1_scores = f1(pd_dataset, predicted)

  return f1_scores

  """avg_f1 = f1(pd_dataset, predicted)

  print(f'average f1: {avg_f1}')"""

  #confusion(pd_dataset, predicted)

## Final Training and Evaluation

In [ ]:
def create_finetuned_model(bert_dir, alberto_dir):

  dropout_rate = 0.1
  units = 300
  learning_rate = 5e-05

  bert_config = BertConfig.from_pretrained(bert_dir)
  bert_encoder = TFBertModel.from_pretrained(bert_dir, trainable=False, config=bert_config)

  alberto_config = BertConfig.from_pretrained(alberto_dir)
  alberto_encoder = TFAutoModel.from_pretrained(alberto_dir, trainable=False, config=alberto_config)


  input_ids_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_input_ids')
  token_type_ids_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_token_type_ids')
  attention_mask_BERT = layers.Input(shape=(max_length,), dtype=tf.int32, name='bert_attention_mask')
  bert_embedding = bert_encoder(
      input_ids_BERT, token_type_ids=token_type_ids_BERT, attention_mask=attention_mask_BERT)['pooler_output']

  bert_embedding = layers.Dropout(dropout_rate)(bert_embedding)



  input_ids_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_input_ids')
  token_type_ids_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_token_type_ids')
  attention_mask_ALBERTO = layers.Input(shape=(max_length,), dtype=tf.int32, name='alberto_attention_mask')
  alberto_embedding = alberto_encoder(
      input_ids_ALBERTO, token_type_ids=token_type_ids_ALBERTO, attention_mask=attention_mask_ALBERTO)['pooler_output']

  alberto_embedding = layers.Dropout(dropout_rate)(alberto_embedding)


  embedding = layers.Concatenate()([bert_embedding, alberto_embedding])


  gender = layers.Dense(units=units)(embedding)
  gender = layers.Activation("relu")(gender)
  gender = layers.Dropout(dropout_rate)(gender)

  ideology = layers.Dense(units=units)(embedding)
  ideology = layers.Activation("relu")(ideology)
  ideology = layers.Dropout(dropout_rate)(ideology)

  multi_ideology = layers.Dense(units=units)(embedding)
  multi_ideology = layers.Activation("relu")(multi_ideology)
  multi_ideology = layers.Dropout(dropout_rate)(multi_ideology)

  gender_output = layers.Dense(1,  activation='sigmoid', name = 'gender')(gender)
  ideology_binary_output = layers.Dense(1,  activation='sigmoid', name = 'ideology_binary')(ideology)
  ideology_multiclass_output = layers.Dense(4,  activation='softmax', name = 'ideology_multiclass')(multi_ideology)

  input_dict = {'bert_input_ids': input_ids_BERT, 'bert_token_type_ids': token_type_ids_BERT, 'bert_attention_mask': attention_mask_BERT,
                'alberto_input_ids': input_ids_ALBERTO, 'alberto_token_type_ids': token_type_ids_ALBERTO, 'alberto_attention_mask': attention_mask_ALBERTO}

  model = keras.Model(
      inputs=input_dict,
      #inputs=[input_ids_BERT, token_type_ids_BERT, attention_mask_BERT, input_ids_ALBERTO, token_type_ids_ALBERTO, attention_mask_ALBERTO],
      outputs=[gender_output, ideology_binary_output, ideology_multiclass_output]
      )

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

  loss = {'gender': 'binary_crossentropy', 'ideology_binary': 'binary_crossentropy', 'ideology_multiclass': 'sparse_categorical_crossentropy'}

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  return model

In [ ]:
new_model = create_finetuned_model(project_dir + 'italian_bert', project_dir + 'alberto')

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/italian_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/alberto.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 bert_input_ids (InputLayer)    [(None, 250)]        0           []                               
                                                                                                  
 bert_attention_mask (InputLaye  [(None, 250)]       0           []                               
 r)                                                                                               
                                                                                                  
 bert_token_type_ids (InputLaye  [(None, 250)]       0           []                               
 r)                                                                                               
                                                                                              

In [ ]:
checkpoint_filepath = project_dir+'bert_alberto_training'

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


In [ ]:
history = new_model.fit(tf_datasets['train'],
          epochs = 4,
          verbose = 1,
          batch_size = 32,
          validation_data = tf_datasets['valid'],
          callbacks=[callback, model_checkpoint_callback]
                    )

In [ ]:
best_model = create_finetuned_model(project_dir + 'italian_bert', project_dir + 'alberto')

best_model.load_weights(checkpoint_filepath)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/italian_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at gdrive/My Drive/HLT_proj/alberto.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
evaluate_model(best_model, tf_datasets['valid'], data['bert']['valid'].copy())

75/75 [==============================] - 85s 1s/step
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       692
           1       0.85      0.90      0.87       406

    accuracy                           0.90      1098
   macro avg       0.89      0.90      0.90      1098
weighted avg       0.91      0.90      0.90      1098

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       612
           1       0.98      0.93      0.96       486

    accuracy                           0.96      1098
   macro avg       0.97      0.96      0.96      1098
weighted avg       0.96      0.96      0.96      1098

              precision    recall  f1-score   support

           0       0.88      0.93      0.90       137
           1       0.96      0.93      0.94       376
           2       0.94      0.97      0.95       475
           3       0.89      0.81      0.85       110

    accuracy         

In [ ]:
f1_scores = evaluate_model(best_model, tf_datasets['test'], data['bert']['test'].copy())

              precision    recall  f1-score   support

           0       0.87      0.86      0.87       318
           1       0.68      0.70      0.69       135

    accuracy                           0.81       453
   macro avg       0.78      0.78      0.78       453
weighted avg       0.82      0.81      0.82       453

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       248
           1       0.96      0.78      0.86       205

    accuracy                           0.89       453
   macro avg       0.90      0.88      0.88       453
weighted avg       0.89      0.89      0.88       453

              precision    recall  f1-score   support

           0       0.84      0.69      0.76       100
           1       0.86      0.81      0.84       154
           2       0.68      0.95      0.79       148
           3       0.70      0.27      0.39        51

    accuracy                           0.77       453
   macro avg       0